# 批归一化（Batch Normalization）技术详解

## 一、概述
批归一化（Batch Normalization，简称BatchNorm）是深度学习领域的重要技术之一，由Sergey Ioffe和Christian Szegedy于2015年提出。该技术通过规范化神经网络中间层的输入分布，显著提升了深度神经网络的训练速度和模型稳定性。

## 二、核心原理
### 1. 基本公式

#### 输入数据格式
- 矩阵形状：`(B, D)`
- B = Batch Size（样本数量）
- D = Feature Dimension（特征维度）

#### 计算步骤分解

### 1. 计算特征维度上的统计量
对每个特征维度**独立计算**批统计量：

$\mu_d = \frac{1}{B}\sum_{b=1}^B x_{b,d} \quad \forall d \in [1,D]$

$\sigma_d^2 = \frac{1}{B}\sum_{b=1}^B (x_{b,d}-\mu_d)^2 \quad \forall d \in [1,D]$

### 2. 归一化

$y_b = \frac{x_b-\mu}{\sqrt{\sigma^2+\epsilon}} \quad \forall b \in [1,B]$

其中，$\epsilon$是一个很小的常数，防止分母为0。


In [8]:
import numpy as np
feature_vector = np.random.rand(4, 5) # 初始化一个4x5的随机特征向量，4是batch_size，5是特征维度
print("以下是初始化的特征向量：")
print(feature_vector)

# 用torch.nn.BatchNorm1d对特征进行归一化
import torch.nn as nn
import torch

batch_norm = nn.BatchNorm1d(5) # 实例化BatchNorm1d，输入特征维度为5
feature_vector_tensor = torch.tensor(feature_vector,dtype=torch.float) # 将numpy数组转换为tensor
torch_normalized_feature_vector = batch_norm(feature_vector_tensor) # 归一化特征
print("以下是用torch.nn.BatchNorm1d归一化后的特征向量：")
print(torch_normalized_feature_vector) # 输出归一化后的特征向量

# 用for 循环、mean和var实现归一化
normalized_feature_vector = []
for channel in range(5):
    sum=0
    for batch_idx in range(4):
        sum += feature_vector[batch_idx][channel]
    mean = sum/4
    var = 0
    for batch_idx in range(4):
        var += (feature_vector[batch_idx][channel]-mean)**2
    var = var/4
    std = np.sqrt(var)
    normalized_channel = (feature_vector[:,channel]-mean)/std
    normalized_feature_vector.append(normalized_channel)
for_normalized_feature_vector = np.array(normalized_feature_vector).T
print("以下是用for 循环、mean和var实现归一化后的特征向量：")
print(for_normalized_feature_vector) # 输出归一化后的特征向量

以下是初始化的特征向量：
[[0.13443642 0.38237412 0.43459833 0.5555555  0.46544354]
 [0.73888243 0.76263731 0.2219813  0.65330424 0.2280716 ]
 [0.16306301 0.62442583 0.54017516 0.64500589 0.29199674]
 [0.9398204  0.87478007 0.10693309 0.75469117 0.82808138]]
以下是用torch.nn.BatchNorm1d归一化后的特征向量：
tensor([[-1.0196, -1.5167,  0.6368, -1.3680,  0.0517],
        [ 0.6942,  0.5529, -0.6090,  0.0165, -0.9665],
        [-0.9385, -0.1993,  1.2554, -0.1010, -0.6923],
        [ 1.2639,  1.1632, -1.2831,  1.4526,  1.6072]],
       grad_fn=<NativeBatchNormBackward0>)
以下是用for 循环、mean和var实现归一化后的特征向量：
[[-1.01966566 -1.51693113  0.63686758 -1.36941136  0.05167255]
 [ 0.69420696  0.55294339 -0.60911539  0.01651852 -0.96662445]
 [-0.93849655 -0.19937877  1.25557123 -0.10113964 -0.69239332]
 [ 1.26395525  1.16336652 -1.28332342  1.45403248  1.60734521]]


## 三、理论推导之后的代码实现

In [2]:
import torch
import torch.nn as nn
import numpy as np

# 参数设置
input_dim = 3       # 输入特征维度（身高、体重、肺活量）
hidden_dim = 64     # 隐藏层维度
num_samples = 100   # 减少采样次数以加快计算
batch_size = 128    # 添加批处理支持
epsilon = 1e-5      # 扰动幅度
dataset_size = 1000 # 训练集大小

# 生成符合要求的训练数据
torch.manual_seed(42)

# 生成特征数据 (1000×3)
height = 1.2 + (2.0 - 1.2) * torch.rand(dataset_size, 1)        # 1.2-2.0米
weight = 30 + (120 - 30) * torch.rand(dataset_size, 1)          # 30-120千克
vital_capacity = 2000 + (5000 - 2000) * torch.rand(dataset_size, 1)  # 2000-5000毫升
x = torch.cat([height, weight, vital_capacity], dim=1)

# 生成目标分数（基于非线性关系+噪声）
y = (height * 35 + torch.log(weight) * 15 + vital_capacity * 0.008)  # 基础分
y += torch.randn_like(y) * 10  # 添加噪声
y = torch.clamp(y, 0, 100)     # 裁剪到0-100分

# 定义带BN的模型
class BNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn(x)
        x = self.relu(x)
        return self.fc2(x)

# 定义不带BN的模型
class NoBNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        return self.fc2(x)

# 初始化模型
bn_model = BNModel()
no_bn_model = NoBNModel()

# 改进的BN预热（使用数据加载器）
class TensorDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

dataset = TensorDataset(x, y)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 预热BN层（模拟真实训练）
bn_model.train()
for _ in range(50):  # 50个epoch的预热
    for batch_x, _ in loader:
        bn_model(batch_x)
bn_model.eval()

# 优化的L估计函数（支持批处理）
def estimate_L(model, x, y):
    model.zero_grad()
    params = list(model.parameters())
    originals = [p.data.clone() for p in params]
    L = []
    
    # 使用相同随机种子保证扰动一致性
    torch.manual_seed(42)
    
    for _ in range(num_samples):
        # 生成参数扰动
        deltas = [torch.randn_like(p) * epsilon for p in params]
        
        # 应用扰动并计算梯度
        for p, delta in zip(params, deltas):
            p.data += delta
        
        y_pred = model(x)
        loss = ((y_pred - y)**2).mean()
        loss.backward()
        grads1 = [p.grad.clone() for p in params]
        model.zero_grad()
        
        # 恢复参数并计算原始梯度
        for p, orig in zip(params, originals):
            p.data = orig
        
        y_pred = model(x)
        loss = ((y_pred - y)**2).mean()
        loss.backward()
        grads2 = [p.grad.clone() for p in params]
        model.zero_grad()
        
        # 计算范数
        grad_diff = sum(torch.norm(g1 - g2, p=2).item()**2 for g1, g2 in zip(grads1, grads2))**0.5
        delta_norm = sum(torch.norm(d, p=2).item()**2 for d in deltas)**0.5
        
        if delta_norm > 1e-7:
            L.append(grad_diff / delta_norm)
    
    return np.median(L), np.max(L)

# 计算结果
bn_median, bn_max = estimate_L(bn_model, x, y)
no_bn_median, no_bn_max = estimate_L(no_bn_model, x, y)

print("="*50)
print(f"[BN模型] Lipschitz常数中位数: {bn_median:.2f}, 最大值: {bn_max:.2f}")
print(f"[无BN模型] Lipschitz常数中位数: {no_bn_median:.2f}, 最大值: {no_bn_max:.2f}")
print("="*50)

# 特征标准化情况对比
print("\n特征统计量对比：")
print("原始数据 - 均值:", x.mean(dim=0).detach().numpy())
print("原始数据 - 标准差:", x.std(dim=0).detach().numpy())

# 查看BN层后的激活值统计
bn_output = bn_model.fc1(x)
bn_output = bn_model.bn(bn_output)
print("\nBN层后激活值 - 均值:", bn_output.mean(dim=0).detach().numpy()[:3])
print("BN层后激活值 - 标准差:", bn_output.std(dim=0).detach().numpy()[:3])

[BN模型] Lipschitz常数中位数: 0.00, 最大值: 7799.11
[无BN模型] Lipschitz常数中位数: 0.00, 最大值: 2415774.36

特征统计量对比：
原始数据 - 均值: [1.5890496e+00 7.5324074e+01 3.4941802e+03]
原始数据 - 标准差: [2.3224361e-01 2.5581488e+01 8.5197577e+02]

BN层后激活值 - 均值: [ 0.00576357  0.00562656 -0.00516746]
BN层后激活值 - 标准差: [1.0035264 1.0027452 1.0029914]
